<a href="https://colab.research.google.com/github/samsung-goya/google_colab_builder/blob/cm-11.0/Android_Build_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuring build environment

In [ ]:
%%shell

sudo apt update

export REPO=$(mktemp /tmp/repo.XXXXXXXXX) 
curl -o ${REPO} https://storage.googleapis.com/git-repo-downloads/repo
gpg --recv-key 8BB9AD793E8E6153AF0F9A4416530D5E920F5C65
curl -s https://storage.googleapis.com/git-repo-downloads/repo.asc | gpg \
--verify - ${REPO} && install -m 755 ${REPO} /usr/bin/repo

# Colab specific deps
sudo apt install tzdata  libnss3 flex gperf \
 ca-certificates-java wget curl libxml2-utils \
 schedtool libbison-dev bison lib32z1

# CM11 uses Python2
# Due to issues with build system in CM11, replace Python3 with Python2
update-alternatives --install /usr/bin/python python /usr/bin/python2.7 10
update-alternatives --install /usr/bin/python3 python3 /usr/bin/python2.7 3
update-alternatives --set python3 /usr/bin/python2.7

git config --global user.email "fake@email.address"
git config --global user.name "Dummy user"

sudo mount -o mode=1777,nosuid,nodev -t ramfs ramfs /tmp

# Downloading source code

In [ ]:
%%shell

echo You will need to deny with N in this next step
echo You will need to deny with N in this next step
echo You will need to deny with N in this next step
echo You will need to deny with N in this next step
repo init -q --depth=1 -u https://github.com/CyanogenMod/android.git -b cm-11.0
mkdir .repo/local_manifests
git clone https://github.com/samsung-goya/android_platform_manifest.git .repo/local_manifests && rm -rf .repo/local_manifests/.git/
repo sync -c --force-sync --optimized-fetch --no-tags --no-clone-bundle --prune -j12
mkdir keep_potf
mv .repo/potf* .
rm -rf .repo
rm -rf /content/sample_data
rm -rf /content/working_dir

# Patching on the fly
source potf.sh

# Installing Java JDK 6 *(closed source)*

In [ ]:
%%shell

mkdir /content/.jdk_6
cd /content/.jdk_6
wget https://repo.huaweicloud.com/java/jdk/6u45-b06/jdk-6u45-linux-x64.bin
chmod +x jdk-6u45-linux-x64.bin
./jdk-6u45-linux-x64.bin
sudo update-alternatives --install /usr/bin/javac javac /content/.jdk_6/jdk1.6.0_45/bin/javac 1
sudo update-alternatives --set javac /content/.jdk_6/jdk1.6.0_45/bin/javac

sudo update-alternatives --install /usr/bin/java java /content/.jdk_6/jdk1.6.0_45/bin/java 1
sudo update-alternatives --set java /content/.jdk_6/jdk1.6.0_45/bin/java

sudo update-alternatives --install /usr/bin/javaws javaws /content/.jdk_6/jdk1.6.0_45/bin/javaws 1
sudo update-alternatives --set javaws /content/.jdk_6/jdk1.6.0_45/bin/javaws

sudo update-alternatives --install /usr/bin/javadoc javadoc /content/.jdk_6/jdk1.6.0_45/bin/javadoc 1
sudo update-alternatives --set javadoc /content/.jdk_6/jdk1.6.0_45/bin/javadoc

sudo update-alternatives --install /usr/bin/javah javah /content/.jdk_6/jdk1.6.0_45/bin/javah 1
sudo update-alternatives --set javah /content/.jdk_6/jdk1.6.0_45/bin/javah

sudo update-alternatives --install /usr/bin/javap javap /content/.jdk_6/jdk1.6.0_45/bin/javap 1
sudo update-alternatives --set javap /content/.jdk_6/jdk1.6.0_45/bin/javap

sudo update-alternatives --install /usr/bin/jar jar /content/.jdk_6/jdk1.6.0_45/bin/jar 1
sudo update-alternatives --set jar /content/.jdk_6/jdk1.6.0_45/bin/jar

# Building Android

In [ ]:
%%shell

. build/envsetup.sh
lunch cm_goyawifi-userdebug
make -j16 bacon